# Plan and Execute

> - 참고 자료: https://wikidocs.net/270688
> - 관련 논문: https://arxiv.org/abs/2305.04091

Plan-and-Execute 에이전트는 먼저 달성할 목표를 구체적인 단계로 나누어 계획을 수립한 뒤, 그 계획을 순차적으로 실행하며 실행 과정에서 얻은 피드백을 반영해 동적으로 조정하는 구조를 갖추고 있습니다.

[ReAct](https://arxiv.org/abs/2210.03629) 에이전트는 사고와 행동을 매 단계 즉시 교차 실행하며 즉각적 피드백에 대응하는 반면, Plan-and-Execute 에이전트는 전체 목표를 세부 단계별로 계획 수립 후 순차 실행하며 피드백으로 계획을 동적으로 조정합니다.

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("../.env", override=True)


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"
_set_env("OPENAI_API_KEY")

## 에이전트 정의

랭그래프에 사전 정의된 `create_react_agent`를 사용하여 ReAct 에이전트를 생성합니다.

In [11]:
from langgraph.prebuilt import create_react_agent
from langchain_tavily import TavilySearch

web_search_tool = TavilySearch(max_results=3)

agent_executor = create_react_agent(
    model="openai:gpt-4.1-mini",
    tools=[web_search_tool],
    prompt="당신은 언제나 친절하고 능숙하게 지원을 제공하며, 사용자의 목표 달성을 돕는 탁월한 조수입니다.",
)

In [12]:
agent_executor.invoke({"messages": [("user", "랭그래프에 대해서 웹검색 해주세요.")]})

{'messages': [HumanMessage(content='랭그래프에 대해서 웹검색 해주세요.', additional_kwargs={}, response_metadata={}, id='a8421bec-c953-4275-974e-ca91ce7f07c0'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CLy03HjiDdZh8Imec5JEB8rc', 'function': {'arguments': '{"query":"랭그래프","search_depth":"basic"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 1311, 'total_tokens': 1335, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_95d112f245', 'id': 'chatcmpl-CP6IB82y7FkwtTZrdjbnDAeL2nr4L', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bd791a95-611a-4297-8aab-57ba10e74d97-0', tool_calls=[{'name': 'tavily_search', 'args': {'query':

## 상태 정의

In [13]:
from typing import TypedDict, Annotated
import operator


class State(TypedDict):
    input: Annotated[str, "사용자 요청"]
    plan: Annotated[list[str], "현재 계획"]
    past_steps: Annotated[list[tuple], operator.add]
    response: Annotated[str, "최종 응답"]

## 계획 노드

In [ ]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


class Plan(BaseModel):
    """계획 실행을 위한 단계별 분류"""

    steps: Annotated[
        list[str], Field(..., description="각 단계들은 정렬된 순서로 배치되어야 합니다")
    ]


def plan_node(state: State):
    system_prompt = (
        "목표를 달성하기 위해 간단한 단계별 계획을 세우세요."
        "각 단계 작업을 정확히 수행하면 정답이 도출되고 마지막 단계 결과가 최종 답안이 됩니다."
        "누락이나 중복 없이 필요한 정보만 포함하세요."
    )

    llm = ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0,
    ).with_structured_output(Plan)

    response = llm.invoke(
        [
            ("system", system_prompt),
            ("user", state["input"]),
        ]
    )
    return {"plan": response["steps"]}